# Function Description: MultipleLinearRegression

This function performs multiple linear regression and makes predictions based on the model, taking a Pandas DataFrame as input (with the dependent variable as the last column) and returning another Pandas DataFrame as output.

## Input Parameters

- df (pandas.DataFrame): the input dataframe.


- opt (bool): a boolean flag to indicate whether or not to optimize the model (default=False).


## Dependencies

This function requires the Pandas, NumPy, Scipy and sfrancia libraries to be installed.

## Function Steps

**1.** The function starts by importing the required libraries, including a custom function for testing the normality of residuals.

**2.** It then defines a nested function to check for multicollinearity among the independent variables, excluding columns with perfect correlation to other columns in the DataFrame.

**3.** The function calculates the number of observations and variables in the input DataFrame, splitting the predictor and dependent variables, and computing the covariance matrix of the independent variables.

**4.** It obtains the coefficients and intercept of the linear regression model by solving the normal equation and prints them.

**5.** The function calculates the fitted values and R² of the model, and performs an F-test to check the overall significance of the model. If the F-test is significant, the function then performs a series of T-tests to check the significance of each individual predictor variable.

**6.** If the opt parameter of the function is set to True and any of the predictor variables are not significant, the function drops those variables and performs the regression again recursively until all predictor variables are significant or until the maximum number of iterations is reached.

**7.** Finally, the function returns a DataFrame with the original input variables and a new column for predicted values based on the linear regression model.


### Additional Details


- The function checks for multicollinearity among the independent variables by calculating the correlation matrix and excluding columns with perfect correlation (i.e., correlation coefficient of 1).

- The function obtains the coefficients and intercept of the linear regression model by solving the normal equation, which involves inverting a matrix. In practice, this can be computationally intensive for large datasets with many variables.

- The function checks the normality and heteroscedasticity of the residuals using the Shapiro-Francia test and the Breusch-Pagan test, respectively. If the residuals are not normal or heteroscedastic, the function may not be appropriate for the data.


In [1]:
def MultipleLinearRegression(df, opt=False, it=0):
    
    '''
    This takes a Pandas DataFrame as input (requires that the column corresponding to the dependent variable be the
    last column of the DataFrame) and returns another Pandas DataFrame as output, performing multiple linear regression
    and making predictions based on the model. It needs the Pandas, NumPy and Scipy Libraries installed.
    
    The function starts by calculating the number of observations and variables in the input DataFrame,
    splitting the predictor and dependent variables, and computing the covariance matrix of the independent variables.
    It then obtains the coefficients and intercept of the linear regression model by solving the normal equation,
    and prints the intercept and coefficients.

    Next, the function calculates the fitted values and R² of the model, and performs an F-test to check the overall
    significance of the model. If the F-test is significant, the function then performs a series of T-tests to check the
    significance of each individual predictor variable.

    If the "opt" parameter of the function is set to True and any of the predictor variables are not significant,
    the function drops those variables and performs the regression again recursively until all predictor variables
    are significant or until the maximum number of iterations is reached.

    Finally, the function returns a DataFrame with the original input variables and a new column for predicted values based
    on the linear regression model.
    
    Args:
        df (pandas.DataFrame): input dataframe
        opt (bool): whether or not to optimize the model
        
    '''
    # Importing needed libraries
    import pandas as pd
    import numpy as np
    from scipy import stats
    from sfrancia import shapiroFrancia
    
    # Fuction for Multicollinearity Analysis
    def multicollinearity_analysis(df):
        
        '''This function aims to exclude columns with perfect correlation to other columns in the DataFrame.'''
    
        # Correlation Matrix
        df_cor = df.corr()

        # Getting columns with perfect correlation
        perfect_correlation_columns = []
        for col in df_cor.columns:
            for row in df_cor.index:
                if col != row:
                    if df_cor[col].loc[row] == 1:
                        if set([col, row]) not in perfect_correlation_columns:
                            perfect_correlation_columns.append(tuple(set([col, row])))
                            
        # Excluding columns                    
        for tuple_col in list(set(perfect_correlation_columns)):
            if tuple_col[0] in df:
                print(f'The "{tuple_col[0]}" column will be deleted for having a perfect correlation with {tuple_col[1]}.\n')
                df = df.drop(tuple_col[0], 1)
                      
        # Returning treated Dataframe
        return df
    
    # Function to get Intercept and Coefficients
    def get_intercept_and_coef(df):
        
        ''' This function finds the intercept and coefficients values based on the covariance matrix.'''
        
        # Getting the Matrix Shape
        N, p = df.shape
        
        # Computing the covariance matrix
        df_cov = pd.DataFrame(np.cov(df.values.T, ddof=1))
        df_cov = df_cov.drop(df_cov.index.max())
        
        # Extracting A and b matrices
        A = df_cov.iloc[:, 0:p-1].values.astype(float)
        b = df_cov.iloc[:,p-1].values.astype(float)
        
        # Solving for the coefficients
        coef = np.linalg.solve(A,b)
        
        # Computing the intercept
        intercept = df.mean()[len(df.mean())-1]
        for i in range(0, p-1):
            intercept -= df.mean()[i] * coef[i]
          
        # Returning the intercept and coefficients
        return intercept, coef
    
    # Function for measure the Coefficients statistical significance of the coefficients
    def ttest(x_v, N, p, soma_quad_fitted_real, intercept, coef):
        
        '''Calculates the statistical significance of the coefficients in a multiple linear regression model.'''
        
        # Getting the degrees of freedom
        dof2 = N - p
        
        # Initialize lists to store results
        list_stat_t = []
        list_pval = []
        list_significance = []
        list_std_error = []
        
        # Add intercept term to predictor variable array
        X_ = np.concatenate((np.array([1 for x in range(0, len(df))]).reshape(-1,1), x_v), axis=1)
        
        # Calculate variance of the regression coefficients
        sigma_squared_hat = soma_quad_fitted_real / (N - p)
        var_beta_hat = np.linalg.inv(X_.T @ X_) * sigma_squared_hat

        # Calculate T statistic and P-value for the intercept term
        list_std_error.append(var_beta_hat[0, 0] ** 0.5)
        estat_t_alpha = intercept/(var_beta_hat[0, 0] ** 0.5)
        pvalt = stats.t.cdf(-abs(estat_t_alpha), dof2) * 2
        if pvalt < 0.05:
            list_significance.append('y')
        else:
             list_significance.append('n') 
        list_stat_t.append(estat_t_alpha)
        list_pval.append(stats.t.cdf(-abs(estat_t_alpha), dof2) * 2)
        
        # Calculate T statistic, P-value, and standard error for each predictor variable
        for i in range(0,p-1):
            estat_t_beta = coef[i]/(var_beta_hat[i+1, i+1] ** 0.5 )  
            list_std_error.append(var_beta_hat[i+1, i+1]** 0.5)
            pvalt = stats.t.cdf(-abs(estat_t_beta), dof2) * 2
            list_stat_t.append(estat_t_beta)
            list_pval.append(stats.t.cdf(-abs(estat_t_beta), dof2) * 2)
            if pvalt < 0.05:
                list_significance.append('y')
            else:
                list_significance.append('n')
        
        # Returning lists
        return list_std_error, list_stat_t, list_pval, list_significance
    
    def breusch_pagan_test(res, yhat):
        
        '''Perform the Breusch-Pagan Test for Heteroskedasticity.'''
        
        # Creating a DataFrame for the Breusch-Pagan Test
        df_bp = pd.DataFrame({'yhat':yhat, 'resid':res}) 
        
        # Adding the 'up' column
        df_bp['up'] = (np.square(df_bp.resid))/np.sum(((np.square(df_bp.resid))/df_bp.shape[0]))
        
        # Changing the value of p for get the correct intercept and coef
        p = 1
        intercept_bp, coef_bp = get_intercept_and_coef(df_bp[['yhat', 'up']])
        
        # Getting the fitted values of the model and the SQReg
        fitted_values_bp = (df_bp['yhat'].values*coef_bp[0]) + intercept_bp
        soma_quad_fitted_med_bp = ((np.array(fitted_values_bp)-df_bp['up'].mean())**2).sum()
        
        # Getting Chisquare Statistic
        chisq = soma_quad_fitted_med_bp/2
        
        # Printing and returning the results
        print('Breusch Pagan Test for Heteroskedasticity.\n')
        print(f'Chi2: {round(chisq, 4)},\t p-value: {round(stats.chi2.sf(chisq, 1),4)}')
        
        if stats.chi2.sf(chisq, 1) < 0.05:
            print('\nThe data shows evidence of heteroskedasticity.')
        else:
            print('\nThe data does not show evidence of heteroskedasticity.')
        return
        
    # Check input types
    if not isinstance(df, pd.DataFrame):
        raise TypeError("df must be a pandas DataFrame.")
    if not isinstance(opt, bool):
        raise TypeError("opt must be a boolean.")
    if not isinstance(it, int):
        raise TypeError("it must be an integer.")
     
    # Checking Null Values
    null_vals = df.isna().sum()
    if len(null_vals.loc[null_vals>0].index) > 0:
        print(f'NaN values in the columns: {null_vals.loc[null_vals>0].index}')
        return
     
    # Initializing the Informative DataFrame
    df_info = pd.DataFrame(index=['Intercept'])
    
    # Excluding columns with perfect correlation
    df = multicollinearity_analysis(df)  
    
    # Intormative DataFrame
    df_info = pd.DataFrame(index=['intercept'] + list(df.columns[0:-1]), columns=['Estimate', 'Std. Error', 'T statistic', 'P value', 'Sig. at 0.05'])
    
    # DataFrame's Shape
    N, p = df.shape
    
    # Getting Intercept and Coefficients
    intercept, coef = get_intercept_and_coef(df)
    
    # Splitting predictor and dependent
    x_v = np.array(df.iloc[:, 0:p-1])
    y_v = np.array(df.iloc[:, p-1])
    
    # Fitted Values
    fitted_values = (x_v@coef) + intercept
    
    # Getting R²
    residuals = y_v - np.array(fitted_values)
    soma_quad_fitted_med = ((fitted_values-y_v.mean())**2).sum()
    soma_quad_fitted_real = (residuals**2).sum()
    R = soma_quad_fitted_med / (soma_quad_fitted_med + soma_quad_fitted_real)
    
    # T-Test
    list_pval = ttest(x_v, N, p, soma_quad_fitted_real, intercept, coef)
    
    # Filling the Informative DataFrame
    df_info['Estimate'].loc['intercept'] = intercept
    df_info['Estimate'][1:] = coef
    df_info[['Std. Error', 'T statistic', 'P value', 'Sig. at 0.05']] = pd.DataFrame(ttest(x_v, N, p, soma_quad_fitted_real, intercept, coef)).T.values
    
    # Printing the model's overall information
    print(f'R²: {round(R,4)},\tThe Dataframe has {N} registers and {p} columns.\n')
    print(f'F-statistic: {round((soma_quad_fitted_med/(p - 1))/(soma_quad_fitted_real/(N - p)),4)},\tp-value (F-statistic): {round((1-stats.f.cdf((soma_quad_fitted_med/(p - 1))/(soma_quad_fitted_real/(N - p)), p - 1, N - p)),5)}\n______________________________________________________________________\n')
        
    # If there's any p-value smaller than 0.05 and the parameter opt == True the function will drop non significant variables
    if opt == True and max(df_info['P value'][1:]) > 0.05:
        print(df_info)
        print('\n______________________________________________________________________\nExcluding columns not statistically significants.')
        max_index = df_info['P value'].loc[(df_info['P value'] == max(df_info[df_info.index != 'intercept']['P value'])) & (df_info.index != 'intercept')].index[0]
        it +=1
        print(f'Excluded column: {max_index}')
        df = df.drop(max_index,1)
        print(f'______________________________________________________________________\n######################################################################')
        print(f'\t\t\tIteration number {it}')
        print(f'n#####################################################################\n______________________________________________________________________\n')
        df = MultipleLinearRegression(df, it=it, opt=True)
        return df
        
    # Returning the original Dataframe with the predicted values
    else:  
        print(df_info)
        print('\n______________________________________________________________________\n')
        df_final = df
        df_final['fitted_values'] = intercept 
        for i in range(0, len(coef)):
            df_final['fitted_values'] += df_final[df_final.columns[i]] * coef[i]
        
        # Testing the normality of the residuals
        if len(df) < 31:
            # Shapiro-Wilk Test
            stat_shap, p_shap = stats.shapiro(residuals)
            
            # Printing Results
            print('Testing the residuals normality using the Shapiro-Wilk test.')
            print(f'\nW: {round(stat_shap,4)},\t p-value: {round(p_shap,4)}')
        else:
            # Shapiro-Francia Test
            stat_shap, p_shap = shapiroFrancia(residuals)['statistics W'], shapiroFrancia(residuals)['p-value']
            
            # Printing Results
            print('Testing the residuals normality using the Shapiro-Francia test.')
            print(f'\nW: {round(stat_shap,4)},\t p-value: {round(p_shap,4)}')
        if p_shap < 0.05:
            print(f'\nThe residuals do not follow a Normal Distribution.\n______________________________________________________________________\n')
        else:
            print(f'\nThe residuals do follow a Normal Distribution.\n______________________________________________________________________\n')
        
        # Applying the Breusch Pagan test for Heteroskedasticity
        breusch_pagan_test(residuals, fitted_values)
        
        # Returning the DataFrame with the predicted values
        return df_final